A **Trie** (pronounced "try"), or **prefix tree**, is a tree-like data structure that specializes in storing and searching for strings. Each node in the tree represents a single character, and a path from the root to any node represents a prefix of one or more strings stored in the tree.

-----

## Why Is It Called a "Prefix Tree"?

You are correct\! It's called a **prefix tree** because common prefixes are stored only once.

If you store the words "bat" and "batman," you don't store two separate strings. The Trie stores the path `b -> a -> t`. The word "batman" simply extends this *existing* path by adding `m -> a -> n`.

This structure makes prefix-based operations, like "find all words that start with 'ba'," incredibly fast.

-----

## How It Works: Your Example

Let's build a Trie with your words: `cat`, `bat`, `tab`, `batman`. We'll use `*` to mark a node that is the **end of a complete word**.

1.  **Insert "cat"**: `(root) -> c -> a -> t*`
2.  **Insert "bat"**: The root doesn't have a 'b' child, so we add it. `(root) -> b -> a -> t*`
3.  **Insert "tab"**: The root doesn't have a 't' child, so we add it. `(root) -> t -> a -> b*`
4.  **Insert "batman"**: We follow the existing path `b -> a -> t`. Since 't' is a child, we add 'm' to it, then 'a', then 'n'. We mark 'n' as the end of a word.

**The final Trie looks like this:**

```
      (root)
      /  |  \
     c   b   t
     |   |   |
     a   a   a
     |   |   |
     t* t* b*
         |
         m
         |
         a
         |
         n*
```

### Processing Your Queries

Now, let's look for your queries:

  * **Query: "bat"**

    1.  Start at `(root)`. Go to `b`.
    2.  From `b`, go to `a`.
    3.  From `a`, go to `t`.
    4.  Is the `t` node marked as an end-of-word (`*`)? **Yes.**
    5.  **Result: Found.**

  * **Query: "ba"**

    1.  Start at `(root)`. Go to `b`.
    2.  From `b`, go to `a`.
    3.  We are at the end of the query string "ba".
    4.  Is the `a` node marked as an end-of-word? **No.**
    5.  **Result: Not Found** (as a *complete word*, though it exists as a prefix).

  * **Query: "news"**

    1.  Start at `(root)`. Look for a child `n`.
    2.  The root only has children `c`, `b`, and `t`.
    3.  **Result: Not Found.**

  * **Query: "ta"**

    1.  Start at `(root)`. Go to `t`.
    2.  From `t`, go to `a`.
    3.  We are at the end of the query string "ta".
    4.  Is the `a` node marked as an end-of-word? **No.**
    5.  **Result: Not Found.**

The key advantage here is **speed**. Searching for any word of length **M** takes **O(M)** time, regardless of how many words (N) are in your dictionary. This is much better than the alternatives you listed.

-----

## Generic Python Implementation (using a HashMap)

You asked how to implement this using a "hashmap with a tree." In Python, our "hashmap" is just a **dictionary (`{}`)**. We use it within each node to store its children.

This implementation is generic and provides the three most common methods: `insert`, `search` (for exact words), and `starts_with` (for prefixes).

```python
class TrieNode:
    """A node in the Trie structure."""
    def __init__(self):
        # The 'hashmap' storing children nodes.
        # Key: a character (e.g., 'a')
        # Value: another TrieNode
        self.children = {} 
        
        # This is the "mark" you mentioned.
        # True if this node represents the end of a complete word.
        self.is_end_of_word = False

class Trie:
    """The Trie data structure."""
    def __init__(self):
        """Initializes the trie with an empty root node."""
        self.root = TrieNode()

    def insert(self, word: str):
        """
        Inserts a word into the trie.
        Time Complexity: O(M), where M is the length of the word.
        """
        current = self.root
        for char in word:
            # If the character path doesn't exist, create it.
            if char not in current.children:
                current.children[char] = TrieNode()
            # Move down to the next node
            current = current.children[char]
        
        # After the loop, mark the last node as the end of the word.
        current.is_end_of_word = True

    def search(self, word: str) -> bool:
        """
        Searches for an exact, complete word in the trie.
        Time Complexity: O(M), where M is the length of the word.
        """
        current = self.root
        for char in word:
            if char not in current.children:
                # Path doesn't exist
                return False
            current = current.children[char]
        
        # The path exists, but is it a *complete* word?
        return current.is_end_of_word

    def starts_with(self, prefix: str) -> bool:
        """
        Checks if any word in the trie starts with the given prefix.
        Time Complexity: O(M), where M is the length of the prefix.
        """
        current = self.root
        for char in prefix:
            if char not in current.children:
                # Prefix path doesn't exist
                return False
            current = current.children[char]
        
        # The prefix path exists.
        return True

# --- Let's use your example ---
trie = Trie()

# Insert words
words_to_insert = ["cat", "bat", "tab", "batman"]
for word in words_to_insert:
    trie.insert(word)

# Run queries
queries = ["bat", "ba", "news", "ta"]

print("--- Query Results ---")
# Query: "bat" (exact word)
print(f"'bat' (exact search): {trie.search('bat')}") # True

# Query: "ba" (exact word)
print(f"'ba' (exact search):  {trie.search('ba')}")  # False

# Query: "ba" (prefix)
print(f"'ba' (prefix search): {trie.starts_with('ba')}") # True

# Query: "news" (exact word)
print(f"'news' (exact search): {trie.search('news')}") # False

# Query: "ta" (exact word)
print(f"'ta' (exact search):  {trie.search('ta')}")  # False
```

-----

##  What Kind of Problems Can We Solve Using a Trie?

Tries are excellent for any problem involving **string prefixes**.

  * **Autocomplete / Typeahead:** This is the classic use. When you type "bat" into a search bar, the program calls `trie.starts_with("bat")` and then explores all child nodes from there (`...man`) to suggest "batman."
  * **Spell Checkers:** You can store a massive dictionary in a Trie. `trie.search("wrod")` would return `False`, flagging it as a potential misspelling.
  * **Phone Books / Contact Search:** As you mentioned, this is perfect for apps like TrueCaller. You can store "9112345" and "9116789". When a user types "911", the Trie instantly finds all numbers with that prefix. A hashmap could only find an *exact* match for "911".
  * **IP Routing:** Routers use a variation of Tries (often a *Radix Trie*) to store IP address prefixes and find the *longest prefix match* to determine where to send data packets.
  * **Word Games:** Finding all valid words on a Boggle board or finding anagrams.


In [7]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False
class Trie:
    def __init__(self):
        self.root = TrieNode()
    def insert(self, word):
        # Time Complexity: O(M), where M is the length of the word.
        current = self.root
        for char in word:
            if char not in current.children:
                current.children[char] = TrieNode()
            # move to the next node
            current = current.children[char]
        current.is_end_of_word = True
    def search(self, word):
        # Time Complexity: O(M), where M is the length of the word.
        current = self.root
        for char in word:
            if char not in current.children:
                return False
            current = current.children[char]
        # the word does exist but check it is complete word
        return current.is_end_of_word
    def starts_with(self, prefix):
        # Time Complexity: O(M), where M is the length of the word.
        current = self.root
        for char in prefix:
            if char not in current.children:
                return False
            current = current.children[char]
        # the prefix path exists
        return True

In [9]:
trie = Trie()

words_to_insert = ["cat", "bat", "tab", "batman"]
for word in words_to_insert:
    trie.insert(word)

queries = ["bat", "ba", "news", "ta"]

print("--- Query Results ---")
# Query: "bat" (exact word)
print(f"'bat' (exact search): {trie.search('bat')}") # True

# Query: "ba" (exact word)
print(f"'ba' (exact search):  {trie.search('ba')}")  # False

# Query: "ba" (prefix)
print(f"'ba' (prefix search): {trie.starts_with('ba')}") # True

# Query: "news" (exact word)
print(f"'news' (exact search): {trie.search('news')}") # False

# Query: "ta" (exact word)
print(f"'ta' (exact search):  {trie.search('ta')}")  # False

--- Query Results ---
'bat' (exact search): True
'ba' (exact search):  False
'ba' (prefix search): True
'news' (exact search): False
'ta' (exact search):  False


## Bitwise Trie
like for problems like:

- maximum XOR pair,

- count of XORs less than a value,

- minimum XOR of any pair, etc.

In [ ]:
cla